In [1]:
!pip install openai

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 81.9 kB/s  0:00:120.6 kB/s eta 0:00:03:07
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 82.5 kB/s  0:00:23 82.3 kB/s eta 0:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [openai]━━━━ 7/8 [openai]c]


In [2]:
from openai import OpenAI

client = OpenAI(api_key="sk-cd6890b8ac624127af85f7094d7c534d", base_url="https://api.deepseek.com")

In [3]:
# Round 1
messages = [{"role": "user", "content": "9.11 和 9.8，哪个更大？"}]
response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=messages
)

In [4]:
response

ChatCompletion(id='5c499e45-5ee5-4a8d-89e6-083d718df726', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='在数字比较中，9.11 和 9.8 都是小数。9.8 可以表示为 9.80，这样更容易比较：\n\n- 9.11 相当于 9.11\n- 9.8 相当于 9.80\n\n比较小数部分：0.11 和 0.80，显然 0.80 大于 0.11。因此，**9.8 更大**。\n\n所以，9.8 > 9.11。', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning_content='首先，用户的问题是“9.11 和 9.8，哪个更大？”这看起来像是在比较两个数字：9.11 和 9.8。在数学中，比较小数时，我们需要逐位比较。\n\n数字 9.11 和 9.8 都是小数。9.11 表示 9 又 11/100，而 9.8 表示 9 又 8/10，或者 9.80。\n\n为了比较它们，我可以将两个数字对齐到相同的小数位数。9.8 可以写成 9.80，这样更容易与 9.11 比较。\n\n现在，比较 9.11 和 9.80：\n\n- 整数部分都是 9，所以相同。\n\n- 然后比较小数部分：9.11 的小数部分是 0.11，而 9.80 的小数部分是 0.80。\n\n显然，0.80 大于 0.11，因为 80/100 大于 11/100。\n\n所以，9.80 > 9.11，因此 9.8 > 9.11。\n\n用户可能是在测试我是否会被数字的写法迷惑，比如 9.11 可能被误认为是日期（9月11日），但在这里上下文是数字比较，所以应该作为数字处理。\n\n因此，答案应该是 9.8 更大。\n\n最后，我需要用中文回复，因为用户的问题是中文的。'))], created=1761545494, model='deepseek-reasoner', object='chat.completion', se

## 方法一：直接使用强制转换方法

In [5]:
response_to_list = list(response)
response_to_list
select1 = response_to_list[1][1]
print(select1)

[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='在数字比较中，9.11 和 9.8 都是小数。9.8 可以表示为 9.80，这样更容易比较：\n\n- 9.11 相当于 9.11\n- 9.8 相当于 9.80\n\n比较小数部分：0.11 和 0.80，显然 0.80 大于 0.11。因此，**9.8 更大**。\n\n所以，9.8 > 9.11。', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning_content='首先，用户的问题是“9.11 和 9.8，哪个更大？”这看起来像是在比较两个数字：9.11 和 9.8。在数学中，比较小数时，我们需要逐位比较。\n\n数字 9.11 和 9.8 都是小数。9.11 表示 9 又 11/100，而 9.8 表示 9 又 8/10，或者 9.80。\n\n为了比较它们，我可以将两个数字对齐到相同的小数位数。9.8 可以写成 9.80，这样更容易与 9.11 比较。\n\n现在，比较 9.11 和 9.80：\n\n- 整数部分都是 9，所以相同。\n\n- 然后比较小数部分：9.11 的小数部分是 0.11，而 9.80 的小数部分是 0.80。\n\n显然，0.80 大于 0.11，因为 80/100 大于 11/100。\n\n所以，9.80 > 9.11，因此 9.8 > 9.11。\n\n用户可能是在测试我是否会被数字的写法迷惑，比如 9.11 可能被误认为是日期（9月11日），但在这里上下文是数字比较，所以应该作为数字处理。\n\n因此，答案应该是 9.8 更大。\n\n最后，我需要用中文回复，因为用户的问题是中文的。'))]


In [6]:
msg = select1[0].message
content = msg.content
print(f"content: {content}")
print("-*-" * 50)
reasoning_content = msg.reasoning_content
print(f"reasoning_content: {reasoning_content}")

content: 在数字比较中，9.11 和 9.8 都是小数。9.8 可以表示为 9.80，这样更容易比较：

- 9.11 相当于 9.11
- 9.8 相当于 9.80

比较小数部分：0.11 和 0.80，显然 0.80 大于 0.11。因此，**9.8 更大**。

所以，9.8 > 9.11。
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
reasoning_content: 首先，用户的问题是“9.11 和 9.8，哪个更大？”这看起来像是在比较两个数字：9.11 和 9.8。在数学中，比较小数时，我们需要逐位比较。

数字 9.11 和 9.8 都是小数。9.11 表示 9 又 11/100，而 9.8 表示 9 又 8/10，或者 9.80。

为了比较它们，我可以将两个数字对齐到相同的小数位数。9.8 可以写成 9.80，这样更容易与 9.11 比较。

现在，比较 9.11 和 9.80：

- 整数部分都是 9，所以相同。

- 然后比较小数部分：9.11 的小数部分是 0.11，而 9.80 的小数部分是 0.80。

显然，0.80 大于 0.11，因为 80/100 大于 11/100。

所以，9.80 > 9.11，因此 9.8 > 9.11。

用户可能是在测试我是否会被数字的写法迷惑，比如 9.11 可能被误认为是日期（9月11日），但在这里上下文是数字比较，所以应该作为数字处理。

因此，答案应该是 9.8 更大。

最后，我需要用中文回复，因为用户的问题是中文的。


## 方法二：点「.」提取方法

In [7]:
response

ChatCompletion(id='5c499e45-5ee5-4a8d-89e6-083d718df726', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='在数字比较中，9.11 和 9.8 都是小数。9.8 可以表示为 9.80，这样更容易比较：\n\n- 9.11 相当于 9.11\n- 9.8 相当于 9.80\n\n比较小数部分：0.11 和 0.80，显然 0.80 大于 0.11。因此，**9.8 更大**。\n\n所以，9.8 > 9.11。', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning_content='首先，用户的问题是“9.11 和 9.8，哪个更大？”这看起来像是在比较两个数字：9.11 和 9.8。在数学中，比较小数时，我们需要逐位比较。\n\n数字 9.11 和 9.8 都是小数。9.11 表示 9 又 11/100，而 9.8 表示 9 又 8/10，或者 9.80。\n\n为了比较它们，我可以将两个数字对齐到相同的小数位数。9.8 可以写成 9.80，这样更容易与 9.11 比较。\n\n现在，比较 9.11 和 9.80：\n\n- 整数部分都是 9，所以相同。\n\n- 然后比较小数部分：9.11 的小数部分是 0.11，而 9.80 的小数部分是 0.80。\n\n显然，0.80 大于 0.11，因为 80/100 大于 11/100。\n\n所以，9.80 > 9.11，因此 9.8 > 9.11。\n\n用户可能是在测试我是否会被数字的写法迷惑，比如 9.11 可能被误认为是日期（9月11日），但在这里上下文是数字比较，所以应该作为数字处理。\n\n因此，答案应该是 9.8 更大。\n\n最后，我需要用中文回复，因为用户的问题是中文的。'))], created=1761545494, model='deepseek-reasoner', object='chat.completion', se

In [8]:
response.choices

[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='在数字比较中，9.11 和 9.8 都是小数。9.8 可以表示为 9.80，这样更容易比较：\n\n- 9.11 相当于 9.11\n- 9.8 相当于 9.80\n\n比较小数部分：0.11 和 0.80，显然 0.80 大于 0.11。因此，**9.8 更大**。\n\n所以，9.8 > 9.11。', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning_content='首先，用户的问题是“9.11 和 9.8，哪个更大？”这看起来像是在比较两个数字：9.11 和 9.8。在数学中，比较小数时，我们需要逐位比较。\n\n数字 9.11 和 9.8 都是小数。9.11 表示 9 又 11/100，而 9.8 表示 9 又 8/10，或者 9.80。\n\n为了比较它们，我可以将两个数字对齐到相同的小数位数。9.8 可以写成 9.80，这样更容易与 9.11 比较。\n\n现在，比较 9.11 和 9.80：\n\n- 整数部分都是 9，所以相同。\n\n- 然后比较小数部分：9.11 的小数部分是 0.11，而 9.80 的小数部分是 0.80。\n\n显然，0.80 大于 0.11，因为 80/100 大于 11/100。\n\n所以，9.80 > 9.11，因此 9.8 > 9.11。\n\n用户可能是在测试我是否会被数字的写法迷惑，比如 9.11 可能被误认为是日期（9月11日），但在这里上下文是数字比较，所以应该作为数字处理。\n\n因此，答案应该是 9.8 更大。\n\n最后，我需要用中文回复，因为用户的问题是中文的。'))]

In [9]:
response.choices[0]

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='在数字比较中，9.11 和 9.8 都是小数。9.8 可以表示为 9.80，这样更容易比较：\n\n- 9.11 相当于 9.11\n- 9.8 相当于 9.80\n\n比较小数部分：0.11 和 0.80，显然 0.80 大于 0.11。因此，**9.8 更大**。\n\n所以，9.8 > 9.11。', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning_content='首先，用户的问题是“9.11 和 9.8，哪个更大？”这看起来像是在比较两个数字：9.11 和 9.8。在数学中，比较小数时，我们需要逐位比较。\n\n数字 9.11 和 9.8 都是小数。9.11 表示 9 又 11/100，而 9.8 表示 9 又 8/10，或者 9.80。\n\n为了比较它们，我可以将两个数字对齐到相同的小数位数。9.8 可以写成 9.80，这样更容易与 9.11 比较。\n\n现在，比较 9.11 和 9.80：\n\n- 整数部分都是 9，所以相同。\n\n- 然后比较小数部分：9.11 的小数部分是 0.11，而 9.80 的小数部分是 0.80。\n\n显然，0.80 大于 0.11，因为 80/100 大于 11/100。\n\n所以，9.80 > 9.11，因此 9.8 > 9.11。\n\n用户可能是在测试我是否会被数字的写法迷惑，比如 9.11 可能被误认为是日期（9月11日），但在这里上下文是数字比较，所以应该作为数字处理。\n\n因此，答案应该是 9.8 更大。\n\n最后，我需要用中文回复，因为用户的问题是中文的。'))

In [10]:
response.choices[0].message

ChatCompletionMessage(content='在数字比较中，9.11 和 9.8 都是小数。9.8 可以表示为 9.80，这样更容易比较：\n\n- 9.11 相当于 9.11\n- 9.8 相当于 9.80\n\n比较小数部分：0.11 和 0.80，显然 0.80 大于 0.11。因此，**9.8 更大**。\n\n所以，9.8 > 9.11。', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning_content='首先，用户的问题是“9.11 和 9.8，哪个更大？”这看起来像是在比较两个数字：9.11 和 9.8。在数学中，比较小数时，我们需要逐位比较。\n\n数字 9.11 和 9.8 都是小数。9.11 表示 9 又 11/100，而 9.8 表示 9 又 8/10，或者 9.80。\n\n为了比较它们，我可以将两个数字对齐到相同的小数位数。9.8 可以写成 9.80，这样更容易与 9.11 比较。\n\n现在，比较 9.11 和 9.80：\n\n- 整数部分都是 9，所以相同。\n\n- 然后比较小数部分：9.11 的小数部分是 0.11，而 9.80 的小数部分是 0.80。\n\n显然，0.80 大于 0.11，因为 80/100 大于 11/100。\n\n所以，9.80 > 9.11，因此 9.8 > 9.11。\n\n用户可能是在测试我是否会被数字的写法迷惑，比如 9.11 可能被误认为是日期（9月11日），但在这里上下文是数字比较，所以应该作为数字处理。\n\n因此，答案应该是 9.8 更大。\n\n最后，我需要用中文回复，因为用户的问题是中文的。')

In [11]:
msg = response.choices[0].message
content = msg.content
print(f"content: {content}")
print("-*-" * 10)

reasoning_content = msg.reasoning_content
print(f"reasoning_content: {reasoning_content}")

content: 在数字比较中，9.11 和 9.8 都是小数。9.8 可以表示为 9.80，这样更容易比较：

- 9.11 相当于 9.11
- 9.8 相当于 9.80

比较小数部分：0.11 和 0.80，显然 0.80 大于 0.11。因此，**9.8 更大**。

所以，9.8 > 9.11。
-*--*--*--*--*--*--*--*--*--*-
reasoning_content: 首先，用户的问题是“9.11 和 9.8，哪个更大？”这看起来像是在比较两个数字：9.11 和 9.8。在数学中，比较小数时，我们需要逐位比较。

数字 9.11 和 9.8 都是小数。9.11 表示 9 又 11/100，而 9.8 表示 9 又 8/10，或者 9.80。

为了比较它们，我可以将两个数字对齐到相同的小数位数。9.8 可以写成 9.80，这样更容易与 9.11 比较。

现在，比较 9.11 和 9.80：

- 整数部分都是 9，所以相同。

- 然后比较小数部分：9.11 的小数部分是 0.11，而 9.80 的小数部分是 0.80。

显然，0.80 大于 0.11，因为 80/100 大于 11/100。

所以，9.80 > 9.11，因此 9.8 > 9.11。

用户可能是在测试我是否会被数字的写法迷惑，比如 9.11 可能被误认为是日期（9月11日），但在这里上下文是数字比较，所以应该作为数字处理。

因此，答案应该是 9.8 更大。

最后，我需要用中文回复，因为用户的问题是中文的。


In [12]:
# Round 2
messages.append({'role': 'assistant', 'content': content})  # 把大模型的回答，添加到 messages
# messages.append({'role': 'user', 'content': "How many Rs are there in the word 'strawberry'?"})
# messages.append({'role': 'user', 'content': "我上一个问题是什么？原问题回答我！"})
messages.append({'role': 'user', 'content': "我的第一个问题是什么？原问题回答我！"})
response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=messages
)
print(messages)

[{'role': 'user', 'content': '9.11 和 9.8，哪个更大？'}, {'role': 'assistant', 'content': '在数字比较中，9.11 和 9.8 都是小数。9.8 可以表示为 9.80，这样更容易比较：\n\n- 9.11 相当于 9.11\n- 9.8 相当于 9.80\n\n比较小数部分：0.11 和 0.80，显然 0.80 大于 0.11。因此，**9.8 更大**。\n\n所以，9.8 > 9.11。'}, {'role': 'user', 'content': '我的第一个问题是什么？原问题回答我！'}]


In [13]:
from pprint import pprint

pprint(messages)

[{'content': '9.11 和 9.8，哪个更大？', 'role': 'user'},
 {'content': '在数字比较中，9.11 和 9.8 都是小数。9.8 可以表示为 9.80，这样更容易比较：\n'
             '\n'
             '- 9.11 相当于 9.11\n'
             '- 9.8 相当于 9.80\n'
             '\n'
             '比较小数部分：0.11 和 0.80，显然 0.80 大于 0.11。因此，**9.8 更大**。\n'
             '\n'
             '所以，9.8 > 9.11。',
  'role': 'assistant'},
 {'content': '我的第一个问题是什么？原问题回答我！', 'role': 'user'}]


In [14]:
response

ChatCompletion(id='133940fd-9d8a-412d-9af3-90efeb4b9786', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='您的第一个问题是：  \n**“9.11 和 9.8，哪个更大？”**', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning_content='嗯，用户突然问“我的第一个问题是什么？原问题回答我！”，看起来有点着急或者想确认之前的对话内容。首先我得回想一下之前的对话历史，用户一开始确实问了“9.11 和 9.8，哪个更大？”，然后我详细解释了比较过程，最后给出答案9.8更大。\n\n用户现在可能是在测试我是否记得之前的互动，或者他们自己忘记之前的问题了。也有可能是想确认我的回答是否一致，避免混淆。考虑到用户重复强调“原问题”，可能他们需要明确的引用，而不仅仅是总结。\n\n深层需求可能是用户对之前的解释还有疑问，或者想再次确认答案的正确性。也许他们遇到别人有不同的观点，想核实一下。另外，用户可能希望得到更简洁直接的回复，不需要额外解释，所以这次回答要直接引用原问题，避免添加新内容。\n\n得确保回复准确，直接复制原问题，并且保持友好，避免让用户觉得我在回避问题。同时保持语气耐心，毕竟重复提问可能意味着用户需要更清晰的确认。'))], created=1761545512, model='deepseek-reasoner', object='chat.completion', service_tier=None, system_fingerprint='fp_ffc7281d48_prod0820_fp8_kvcache', usage=CompletionUsage(completion_tokens=236, prompt_tokens=135, total_tokens=371, completion_tokens_details=CompletionTokensDetails(ac

import notebook
print(notebook.__version__)

In [15]:
import openai
print(openai.__version__)

2.6.1


In [16]:
a = 1
a

1

In [17]:
# Round 1
messages = [{"role": "user", "content": "9.11 和 9.8，哪个更大？"}]
response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=messages,
    stream=True
)

# 方法一：

```python
# Round 1
messages = [{"role": "user", "content": "9.11 和 9.8，哪个更大？"}]
response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=messages,
    stream=True
)
for chunk in response:
    print(chunk.choices)
```

In [18]:
# # 方法二：
# from copy import deepcopy
# copy_response = response
# for chunk in response:
#     print(chunk.choices)

In [19]:
reasoning_content = ""
content = ""

for chunk in response:
    if chunk.choices[0].delta.reasoning_content:
        reasoning_content += chunk.choices[0].delta.reasoning_content
    else:
        content += chunk.choices[0].delta.content

TypeError: can only concatenate str (not "NoneType") to str

In [ ]:


# # Round 2
# messages.append({"role": "assistant", "content": content})
# messages.append({'role': 'user', 'content': "How many Rs are there in the word 'strawberry'?"})
# response = client.chat.completions.create(
#     model="deepseek-reasoner",
#     messages=messages,
#     stream=True
# )

## 使用 API 实现多轮对话，并且拥有记忆功能。

In [ ]:
from openai import OpenAI
client = OpenAI(api_key="sk-cd6890b8ac624127af85f7094d7c534d", base_url="https://api.deepseek.com")

In [ ]:
messages = [] # 用来存储历史对话，后期学完文件读取，可以考虑持久化存储！

In [ ]:
while True:
    prompt = input("Enter your prompt: ")
    if prompt.lower() in ['quit', 'q', 'exit', '退出']:
        break
    messages.append({'role': 'user', 'content': prompt})
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages
    )
    assistant_content = response.choices[0].message.content
    messages.append({"role": "assistant", "content": assistant_content})
    print(f"Message: {assistant_content}")
    

In [ ]:
# 「可选」：角色设定
system_setting1 = {
    "role": "system", 
    "content": "用户如果问你使用的是什么大模型，你就回答：AI悦创编程1v1大模型（bornforthis）"
}
system_setting2 = {
    "role": "system", 
    "content": "全部回答的末尾都附上：欢迎跟随AI悦创学习编程！\n微信：Jiabcdefh\n网站：bornforthis.cn。注意：只需要在回答的末尾加上即可！"
}

messages.append(system_setting1)
messages.append(system_setting2)
while True:
    prompt = input("Enter your prompt: ")
    if prompt.lower() in ['quit', 'q', 'exit']:
        break
    messages.append({'role': 'user', 'content': prompt})
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages
    )
    assistant_content = response.choices[0].message.content
    messages.append({"role": "assistant", "content": assistant_content})
    print(f"Messages: {assistant_content}")

In [ ]:
code_string = '''
import random
from faker import Faker


# —— 简单无色进度条（纯 ASCII，跨平台）——
def hp_bar(cur: int, maxv: int, width: int = 20) -> str:
    """返回形如：[##########----------] 50%  5g0/100 的进度条文本"""
    if maxv <= 0:
        maxv = 1
    cur = max(0, min(cur, maxv))
    ratio = cur / maxv
    filled = int(ratio * width + 0.5)  # 四舍五入
    bar = "#" * filled + "-" * (width - filled)
    return f"[{bar}] {int(ratio * 100):3d}%  {cur}/{maxv}"


class Creature:
    def __init__(self, hp, name):
        self.hp = int(hp)
        self.max_hp = int(hp)  # 记录初始满血
        self.name = name

    def attack(self):
        return random.randint(0, 50)

    def not_dead(self):
        return self.hp > 0

    def being_attack(self, dmg: float):
        """受到伤害（向下取整以避免浮点 HP），并保证 HP 不会掉到负数以下"""
        self.hp = max(0, int(self.hp - dmg))

    def heal_full(self):
        """直接回到初始满血"""
        self.hp = self.max_hp

    def show_status(self):
        print(f"{self.name}'s HP → {hp_bar(self.hp, self.max_hp)}")

fk = Faker(locale='zh_CN')
player = Creature(100, "AI悦创")
enemy = Creature(80, fk.name())

heal_used = False  # 治疗仅限一次
heal_penalty_active = False  # 是否已触发“敌人攻击翻倍”的惩罚

while player.not_dead() and enemy.not_dead():
    player.show_status()
    enemy.show_status()

    # 是否出现治疗提示：仅当当前 HP < 初始 HP 的 50% 且尚未使用
    can_heal_now = (not heal_used) and (player.hp < player.max_hp * 0.5)

    if can_heal_now:
        prompt = 'Attack or Defence or Heal (A/D/H)：'
        valid_inputs = {"A", "D", "H"}
        print('（提示：你现在可以按 H 回满血，仅此一次）')
    else:
        prompt = 'Attack or Defence (A/D)：'
        valid_inputs = {"A", "D"}

    if heal_penalty_active:
        print('【警告】治疗代价生效中：敌人对你的伤害 ×2！')

    user_input = input(prompt).strip().upper()
    while user_input not in valid_inputs:
        user_input = input("输入无效，请重新输入：" + prompt).strip().upper()

    # 敌人选择（对 A/D 有影响；若玩家选择 H，我们让敌人本回合直接攻击）
    enemy_status = ['Attack', 'Defence']
    enemy_choice = random.choices(enemy_status, weights=[0.7, 0.3], k=1)[0]  # 敌人更倾向于攻击


    # 当前敌人伤害倍率（是否翻倍）
    def enemy_mul():
        return 2.0 if heal_penalty_active else 1.0


    if user_input == "H":
        # 只有在 can_heal_now 为 True 时才会进入到这里
        print("你使用了治疗技能！血量已回满。")
        player.heal_full()
        heal_used = True

        # 触发治疗代价：从现在起敌人攻击翻倍（包含本回合的随后的敌人攻击）
        heal_penalty_active = True

        # 敌人回合：直接攻击（先治再挨打）
        raw_enemy_attack_value = enemy.attack()
        damage = raw_enemy_attack_value * enemy_mul()
        print(f"{enemy.name} 攻击了你，造成 {int(damage)} 点伤害！（原始{int(raw_enemy_attack_value)} × 倍率{enemy_mul():.0f}）")
        player.being_attack(damage)

    elif user_input == "A":
        player_attack_coefficient = 1
        if enemy_choice == "Defence":
            print(f"{enemy.name} chose to defend!")
            player_attack_coefficient = 0.5
        else:
            print(f"{enemy.name} chose to attack!")
            raw_enemy_attack_value = enemy.attack()
            damage = raw_enemy_attack_value * enemy_mul()
            print(f"{enemy.name} 对你造成 {int(damage)} 点伤害！（原始{int(raw_enemy_attack_value)} × 倍率{enemy_mul():.0f}）")
            player.being_attack(damage)

        player_attack_value = player.attack()
        enemy.being_attack(player_attack_value * player_attack_coefficient)

    elif user_input == "D":
        # 防御：敌人攻击减伤为 90%，然后再应用翻倍倍率
        raw_enemy_attack_value = enemy.attack()
        damage = raw_enemy_attack_value * 0.1 * enemy_mul()
        print(f"{enemy.name} 攻击了你（被你防住大部分），造成 {int(damage)} 点伤害！"
              f"（原始{int(raw_enemy_attack_value)} × 减伤0.1 × 倍率{enemy_mul():.0f}）")
        player.being_attack(damage)

if player.not_dead():
    print("You Win!")
else:
    print("You Lose!")
'''
messages = []
code_string = "为下面的代码编写一个游戏说明，也就是游戏场景中的规则介绍、操作介绍等。应用在命令行对话游戏中，要贴合、适合终端显示的 Python 字符串" + code_string
messages.append({'role': 'user', 'content': code_string})
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=messages
    )
assistant_content = response.choices[0].message.content
# messages.append({"role": "assistant", "content": assistant_content})
# print(f"Messages: {assistant_content}")
# while True:
#     prompt = input("Enter your prompt: ")
#     if prompt.lower() in ['quit', 'q', 'exit']:
#         break
#     messages.append({'role': 'user', 'content': prompt})
#     response = client.chat.completions.create(
#         model="deepseek-chat",
#         messages=messages
#     )
#     assistant_content = response.choices[0].message.content
#     messages.append({"role": "assistant", "content": assistant_content})
#     print(f"Messages: {assistant_content}")

In [ ]:
messages = []
s1 = "为下面的代码编写一个游戏说明，也就是游戏场景中的规则介绍、操作介绍等。应用在命令行对话游戏中，要贴合、适合终端显示的 Python 字符串"
s2 = "设计时，可以适当的左右并排显示，防止规则过长"
code_string = s1 + s2 + code_string
messages.append({'role': 'user', 'content': code_string})
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=messages
    )
assistant_content = response.choices[0].message.content
print(assistant_content)

In [ ]:
messages = []
s1 = "为下面的代码编写一个游戏说明，也就是游戏场景中的规则介绍、操作介绍等。应用在命令行对话游戏中，要贴合、适合终端显示的 Python 字符串"
s2 = "设计时，可以适当的左右并排显示，注意边框过长，防止规则过长"
code_string = s1 + s2 + code_string
messages.append({'role': 'user', 'content': code_string})
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=messages
    )
assistant_content = response.choices[0].message.content
print(assistant_content)

In [ ]:
string = """
# =================================================================
#                     战斗游戏说明
# =================================================================
# 规则说明           │ 操作说明           │ 特殊机制
# ------------------│-------------------│--------------------------
# • 回合制战斗      │ A - 攻击          │ 治疗限制：
# • HP归零即失败    │ D - 防御          │  • HP<50%时可治疗
# • 敌人AI随机行动  │ H - 治疗(限1次)   │  • 治疗后敌人伤害×2
# • 攻防存在克制    │ 输入不区分大小写   │  
# =================================================================
# 血量显示：[##########----------] 50%  50/100
# =================================================================
"""
messages = []
s1 = "把下面的字符串中的‘竖杠‘进行对齐！’"
code_string = s1 + string
messages.append({'role': 'user', 'content': code_string})
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=messages
    )
assistant_content = response.choices[0].message.content
print(assistant_content)